In [143]:
import jieba
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import csv

In [80]:
def rmpunct(s):
    punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳/﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃\t﹙+﹛﹝（｛“‘-—_…''')
    return ''.join(filter(lambda x: x not in punct, s))

def extractUrl(url):
    if url.split('.').__len__() < 2:
        return ''
    return url.split('.')[1]

def rmstopwords(s):
    stopwords = set('的和呢了吧')
    return ''.join(filter(lambda x: x not in stopwords, s))

def splitwords(s):
    return ' '.join(jieba.cut(s))

In [132]:
item_info = pd.read_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_profile.txt', sep='\t', header=None)
item_info.columns = ['商品标题', '商品url', '商品价格', '商品描述', '品牌', '类目']
#填充缺失值
item_info.商品标题.fillna('', inplace=True)
item_info.商品url.fillna('', inplace=True)
item_info.商品描述.fillna('', inplace = True)
item_info.品牌.fillna('', inplace = True)
item_info.类目.fillna('',inplace = True)
item_info.商品价格.fillna('', inplace = True)
#滤除行
item_info.drop(item_info[item_info.类目.apply(lambda x: x == '')].index, inplace= True)
item_info.drop(item_info[item_info.商品标题.apply(lambda x: x == '')].index, inplace= True)
item_info.drop(item_info[item_info.类目.apply(lambda x: x.split('$').__len__() == 2)].index, inplace=True)

#取特定类目样本
item_list = ['服装配饰', '个护化妆', '鞋']
sub_item = pd.DataFrame()
for i in item_list:
    sub_item = pd.concat([sub_item, item_info[item_info.类目.apply(lambda x: i in x)]])
    
#取样本数大于1000的标签进行分类训练    
sub_item.index = range(len(sub_item))
labels = sub_item.类目.value_counts()[:60]
sub_item = sub_item[sub_item.类目.apply(lambda x: x in labels)]

#去符号，抽rul，处理label，去停止词
sub_item.商品标题 = sub_item.商品标题.apply(lambda x: rmpunct(x))
sub_item.商品url = sub_item.商品url.apply(lambda x: extractUrl(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: rmpunct(x))
sub_item.类目 = sub_item.类目.apply(lambda x: '|'.join(x.split('$')[1:]))

sub_item.商品标题 = sub_item.商品标题.apply(lambda x: rmstopwords(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: rmstopwords(x))

sub_item.商品标题 = sub_item.商品标题.apply(lambda x: splitwords(x))
sub_item.商品描述 = sub_item.商品描述.apply(lambda x: splitwords(x))

sub_item['label'] = sub_item.类目.apply(lambda x: '__label__' + x)

#划分数据集，80%训练集，20%测试
labels = sub_item.label.unique()
item_train = pd.DataFrame()
item_valid = pd.DataFrame()
for l in labels:
    sample_train = sub_item[sub_item.label == l].sample(frac = 0.8)
    sample_valid = sub_item[sub_item.label == l].drop(sample_train.index)
    item_train = pd.concat([item_train, sample_train])
    item_valid = pd.concat([item_valid, sample_valid])
    
item_train = shuffle(item_train)
item_valid = shuffle(item_valid)

item_train[['商品标题', '商品url', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_train_clothes.txt', sep = ' ', header = None, index = None)
item_valid[['商品标题', '商品url', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_valid_clothes.txt', sep = ' ', header = None, index = None)


In [146]:
item_train[['商品标题', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_train_clothes.txt', sep = ' ', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')
item_valid[['商品标题', '商品描述', '品牌', 'label']].to_csv('/home/zhuyuhe/mydata/item_profilt_classification/item_info_fasttext_valid_clothes.txt', sep = ' ', header = None, index = None, quoting = csv.QUOTE_NONE, escapechar = ' ')